In [1]:
from pathlib import Path

import numpy as np
import re
from pprint import pprint

import sys
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

import csv 

In [2]:
from pdf_processing import *

TODO: 
 - check more PDFs for differences in format
 - write documentation
 - locations not included after the date (might need to be taken care of manually)
 - footnotes (like in People_of_Greece.pdf and Senate_Floor_Immigration_Reform.pdf)



DONE: 
 - Q&As / interviews
     - function to count the number of occurrences of "Obama:", "Question:", etc.
   
      
 - locations with multiple comma's (ex. when Washington, D.C. is included) --> might be better to store location as one elements instead of splitting it up, the location can be cleaned up afterwards
     - location is not split up anymore, but kept as a single element (will be easier to clean up afterwards)

In [3]:
# pdfs that have these in the title might have to be excluded: interview, presser, Press_Conference, press, meeting, 
# debate, hall (maybe)
multiple_people_talking = ['Barack_Obama_-_Al-Arabiya_Interview.pdf', 'CGI_2013.pdf', 
                           'Midterm_Elections_Presser_2014.pdf', 'YSEALI_Fellows.pdf', 
                           'PM_Abe_of_Japan_Joint_Presser.pdf', 'Paris_Press_Conference_2015.pdf',
                           'Gun_Violence_Presser.pdf', 'Associated_Press_Luncheon.pdf', 'Final_Press_Conference.pdf',
                           'CNN_Guns_in_America_Town_Hall.pdf', 'START_Treaty_Presser.pdf', 
                           'Midterm_Elections_Presser_2010.pdf', 'Joint_Presser_with_President_Benigno_Aquino.pdf',
                           'Worldwide_Troop_Talk.pdf', 'Post_ASEAN_Presser_2016.pdf', 'Post_G7_Presser_Japan.pdf',
                           'Presser_Unannounced_Heath_Care.pdf', 'Kenya_Civil_Society_Meeting.pdf',
                           'Obama-Romney_-_Second_Live_Debate.pdf', 'Jamaica_Town_Hall.pdf', 'YSEALI_Town_Hall.pdf',
                           'BP_Oil_Spill_Presser.pdf', 'VA_Misconduct_Allegations.pdf', 'ISIL_Pentagon_Update.pdf',
                           'ISIL_Presser_Post_Security_Council_Meeting.pdf']
# maybe: 'Alabama_Tornado_Devestation.pdf', 'NAACP_Coference_2015.pdf'

# the year for Community_College_Plan.pdf is 20105 but should be 2015

Get the path to the directory in which the PDFs are stored.

In [4]:
pdf_dir = Path.cwd().parent / "pdfs"

Filepaths of all the PDFs in the folder `pdf_dir`, can be used to iterate over all the PDFs to store the extracted speeches in a dataframe.

In [5]:
pdfs = list(pdf_dir.glob('*.pdf'))  
print("current number of PDFs:", len(pdfs))

current number of PDFs: 436


Get filepath of the PDF you want to process.

In [6]:
filepath = pdfs[5]
print("File:", str(filepath.parts[-1]))

File: White_House_Correspondent_Dinner_2013.pdf


Make a PDFHandler object for the given filepath.

In [7]:
pdf = PDFHandler(filepath)

Print the last page of the PDF before it has been processed.

In [8]:
print(pdf.original_page(-1))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
If anyone wonders, for example, whether newspapers are a thing of the past, all you needed 
to do was to pick up or log on to papers like the Boston Globe.  When their communities and 
the wider world needed them most, they were there making sense of events that might at 
first blush seem beyond our comprehension. And that’s what great journalism is, and that’s 
what great journalists do. And that’s why, for example, Pete Williams' new nickname around 
 
the NBC newsroom is "Big Papi."
And in these past few weeks, as I've gotten a chance to meet many of the first responders 
and the police officers and volunteers who raced to help when hardship hits, I was reminded, 
as I'm always reminded when I meet our men and women in uniform, whether they're in war 
theater, or here back home, or at Walter Reed in Bethesda -- I'm reminded that all these 
folks, they don’t do it to be honored, they don’t do it to be celebrated. They do it because t

Print the first page of the PDF before it has been processed.

In [9]:
print(pdf.original_page(0))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
Barack Obama 
White House Correspondents' Dinner Address 
 
delivered 27 April 2013, Washington Hilton Hotel, Washington, D.C.
 
 
AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio 
Thank you. Thank you, everybody.  How do you like my new entrance music?  Rush Limbaugh 
 
warned you about this -- second term, baby.  We’re changing things around here a little bit.
Actually, my advisors were a little worried about the new rap entrance music.  They are a little 
more traditional. They suggested that I should start with some jokes at my own expense, just 
take myself down a peg. I was like, guys, after four and a half years, how many pegs are 
 
there left?
I want to thank the White House Correspondents. Ed, you’re doing an outstanding job. We are 
grateful for -- the great work you’ve done. To all the dignitaries who are here, everybody on 
the dais -- I especially want to say thank you to [General] Ray Odierno, w

Define a regular expression to get the date, location, and content of the speech. Extract the entire speech from the PDF.

In [10]:
start = r"^(?:\s*AA *RR\s+mmeerriiccaann *hheettoorriicc\.\.ccoomm|\s*AmericanRhetoric\.com|\s*AAmmeerriiccaannRRhheettoorriicc\.\.ccoomm)?"

datemarker =r"^(delivered|First *Broadcast|Uploaded[ a-z]*)?"
date = r"(?P<day>[0-9]{1,2}) *(?P<mon>[a-z]+) *(?P<year>[0-9]{2,})"
loc = r"(, *(?P<location>.+?))?"
auth = r"(?:.?AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio( *and edited for continuity)?.?)?"

content = r"(?P<content>.*)"

#footnotes = r"^(([1-9][0-9]*.+?)+\n+)?"
tr = r"(Transcription *by *[a-z.\- ]+\s*)?"
url = r"((Property *of *)?AmericanRhetoric\.com)?"
page = r"(page *[0-9]+\s*$)"

full_pat = start + \
r"(\s+.*?" + datemarker + " *" + date + loc + r"\s+" + auth + r")?" + \
r"\s+" + content + \
r"^(?:\s*" + tr + url + r".*?" + page + r")"
pat = re.compile(full_pat, re.I | re.DOTALL | re.M)

speech = pdf.extract_speech(pat)
print(speech)

Thank you. Thank you, everybody.  How do you like my new entrance music?  Rush Limbaugh 
 
warned you about this -- second term, baby.  Were changing things around here a little bit.
Actually, my advisors were a little worried about the new rap entrance music.  They are a little 
more traditional. They suggested that I should start with some jokes at my own expense, just 
take myself down a peg. I was like, guys, after four and a half years, how many pegs are 
 
there left?
I want to thank the White House Correspondents. Ed, youre doing an outstanding job. We are 
grateful for -- the great work youve done. To all the dignitaries who are here, everybody on 
the dais -- I especially want to say thank you to [General] Ray Odierno, who does outstanding 
 
service on behalf of our country, and all our men and women in uniform every single day.
And of course, our extraordinary First Lady, Michelle Obama. Everybody loves Michelle.  Shes 
on the cover of Vogue, high poll numbers. But dont worr

Print the relevant info of the PDF.

In [11]:
pdf.print_info()

Title: White_House_Correspondent_Dinner_2013
Number of pages: 6
Date: 27 April 2013
Location: Washington Hilton Hotel, Washington, D.C.


Check whether there are multiple speakers. (If there is a high "Obama:" count, this might indicate he is answering questions, i.e., it is an interview, press conference, debate,...)

In [12]:
count = ('obama:', 'president:', 'question:', 'audience:', 'member:')
counts, speakers = pdf.multiple_speakers(speech, count)
print(counts)
print()
pprint(speakers)

{'obama:': 0, 'president:': 0, 'question:': 0, 'audience:': 0, 'member:': 0}

['right:', 'dilemma:', '2:', 'confess:', 'simple:']


Replace substrings.

In [13]:
rep_old = [r'"', r'’’', r'‘‘', r'“', r'”', r'[sic]', r";", r":"]
rep_new = [r'' , r''  , r''  , r'' , r'' , r' '    , r'' , r'' ]

clean_speech = pdf.substring_replace(speech, rep_old, rep_new)

Replace or delete some characters to clean the speech (only the ones that need to be matched with a regular expression).

In [14]:
re_old = [r'-{2,}', r'\.{2,}', r'\s+', r'\[(Source:)?\s*http.*?\]', r'[\[\]\(\)]']
re_new = [r' '    , r' '     , r' '  , r''                        , r''          ]

clean_speech = pdf.match_replace(clean_speech, re_old, re_new)
print(clean_speech)

Thank you. Thank you, everybody. How do you like my new entrance music? Rush Limbaugh warned you about this second term, baby. Were changing things around here a little bit. Actually, my advisors were a little worried about the new rap entrance music. They are a little more traditional. They suggested that I should start with some jokes at my own expense, just take myself down a peg. I was like, guys, after four and a half years, how many pegs are there left? I want to thank the White House Correspondents. Ed, youre doing an outstanding job. We are grateful for the great work youve done. To all the dignitaries who are here, everybody on the dais I especially want to say thank you to General Ray Odierno, who does outstanding service on behalf of our country, and all our men and women in uniform every single day. And of course, our extraordinary First Lady, Michelle Obama. Everybody loves Michelle. Shes on the cover of Vogue, high poll numbers. But dont worry I recently got my own magazi

Output to csv file.

In [15]:
csv_filename = Path.cwd().parent / "speeches_csv" / "speeches.csv"

fields = ["title", "pages", "date", "location", "highest_speaker_count", "content"]

# use 'a+' instead of 'w' if you want to append at the bottom of the file instead of replacing the file
with open(csv_filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL) 
        
    csvwriter.writerow(fields) 
        
    for i in range(10):  # for i in range(len(pdfs)):
        filepath = pdfs[i]
        pdf = PDFHandler(filepath)
        row = np.array(pdf.full_extract(pat, count, rep_old, rep_new, re_old, re_new))
        csvwriter.writerow(row)
        
        print("pdf", i+1)
        print(row[:-1])
        print(row[-1][:100])
        print()

pdf 1
['CGI_2013' '19' '24 September 2013'
 'Sheraton New York Hotel and Towers, New York, New York' '10']
Hillary Clinton Thank you very much. I have the great pleasure to introduce our next two speakers, w

pdf 2
['Prayer_Breakfast_2016' '7' '4 February 2016'
 'Washington Hilton, Washington, D.C.' '0']
Well, good morning. Giving all praise and honor to God for bringing us together here this morning. I

pdf 3
['Security_Team_Announcement' '5' '1 December 2008' 'Chicago, Illinois'
 '0']
Good morning, everybody. I hope you all had a wonderful Thanksgiving. Last week, we announced our ec

pdf 4
['Cairo_University' '14' '4 June 2009' 'Cairo, Egypt' '0']
Thank you so much. Good afternoon. I am honored to be in the timeless city of Cairo, and to be hoste

pdf 5
['Umpqua_Community_College_Shootings' '4' '1 October 2015'
 'Washington, D.C.' '0']
Theres been another mass shooting in America this time, in a community college in Oregon. That means

pdf 6
['White_House_Correspondent_Dinner_2013' 